In [58]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time 

In [59]:
driver = webdriver.Chrome('/Users/utkarsh/PycharmProjects/chromedriver')
driver.implicitly_wait(30)

In [60]:
driver.set_page_load_timeout(10)

In [61]:
#driver.get('https://etrain.info/in?PAGE=runningHistory')
# get the search textbox
#search_field = driver.find_element_by_id("runhistrn")
#search_field2 = driver.find_element_by_id("bartrainid")
#search_field.clear()

# enter search keyword and submit
#search_field.send_keys("12557")
#search_field2.send_keys("12557")
#driver.find_element_by_css_selector('.runhisbtngo.donthide.btn.btn-primary').click()

#from running status
driver.get('https://runningstatus.in/history')
driver.execute_script("document.forms['history_form'].childNodes[3].children[0].value=12557")
driver.execute_script("document.getElementById('rdatesel').value='seldates'")
driver.execute_script("showhistoryseldates();")
driver.execute_script("document.getElementById('fdate').value='02-12-2019'")
driver.execute_script("document.getElementById('tdate').value='12-01-2020'")
driver.execute_script("var buttons = document.getElementsByClassName('btn btn-primary');\
Array.prototype.forEach.call(buttons, function(btn, index) \
{if(btn.innerText=='Check History')btn.click();\
});")


In [62]:
# timeout = 3
# try:
#     element_present = EC.presence_of_element_located((By.CLASS_NAME, 'main'))
#     WebDriverWait(driver, timeout).until(element_present)
# except TimeoutException:
#     print("Timed out waiting for page to load")
# finally:
#     print("Page loaded")

In [63]:
import time 
time.sleep(10)
soup_level1=BeautifulSoup(driver.page_source, 'lxml')

In [64]:
table_data = [[cell.text for cell in row("td")]
                         for row in soup_level1("tr")]
#print(table_data)

In [65]:
complete_date_range_data=[]
station_date_range_data=[]
each_date_data={}
each_station_data={}
each_date_count=0
date_start_count=0;
for row in soup_level1("tr"):
    for cell in row('td'):
        for abbr in cell('abbr'):
            if 'station_name' in each_station_data.keys():
                station_date_range_data.append(each_station_data)
                each_station_data={}            
            each_station_data['station_name']=abbr.text
        for small in cell('small',{'class':None}):
            each_station_data['distance_from_Source']=small.text
        for span in cell('span',{'class':'text-danger'}):
            each_station_data['avg_delayed']=span.text
        for small in cell('small'):
            if date_start_count !=0:
                each_date_count+=1
                if each_date_count == 1:
                    each_date_data['departure_delayed']=small.text
                if each_date_count == 2:
                    each_date_data['arrival_delayed']=small.text
                if each_date_count == 3:
                    each_date_data['departure_time']=small.text
                if each_date_count == 4:
                    each_date_data['arrival_time']=small.text
                    complete_date_range_data.append(each_date_data)
                    each_date_data={}
                    each_date_count=0
        for a in cell('a',{'class':'text-info'}):
            date_start_count=1
            each_date_data['Date_of_journey']=a.text
    #print(row('td').find_all('abbr'))
#print(annual_data)


In [66]:
#print(annual_data)

In [67]:
df_station= pd.DataFrame(station_date_range_data)
df_date_range=pd.DataFrame(complete_date_range_data)

In [68]:
display(df_station.head(10))
display(df_date_range.head(100))

,station_name,distance_from_Source,avg_delayed
0,Muzaffarpur,0 KM's,03 Mins
1,Motipur,27 KM's,13 Mins
2,Mehsi,40 KM's,20 Mins
3,Chakia,48 KM's,27 Mins
4,Bapudm Motihari,80 KM's,36 Mins
5,Sagauli,101 KM's,46 Mins
6,Bettiah,123 KM's,56 Mins
7,Narkatiaganj,160 KM's,47 Mins
8,Harinagar,175 KM's,56 Mins
9,Bagaha,201 KM's,01 Hr 07 Mins


,Date_of_journey,departure_delayed,arrival_delayed,departure_time,arrival_time
0,"02 Dec, 2019",No Delay,10 Hrs 50 Mins Late,11:35 AM,06:30 PM
1,"03 Dec, 2019",No Delay,05 Mins Late,11:35 AM,07:45 AM
2,"04 Dec, 2019",No Delay,02 Hrs 10 Mins Late,11:35 AM,09:50 AM
3,"05 Dec, 2019",01 Min Late,03 Hrs 13 Mins Late,11:36 AM,10:53 AM
4,"06 Dec, 2019",No Delay,02 Hrs 02 Mins Late,11:35 AM,09:42 AM
5,"07 Dec, 2019",No Delay,02 Hrs 30 Mins Late,11:35 AM,10:10 AM
6,"08 Dec, 2019",No Delay,02 Hrs 55 Mins Late,11:35 AM,10:35 AM
7,"11 Dec, 2019",No Delay,01 Hr 00 Min Late,11:35 AM,08:40 AM
8,"12 Dec, 2019",01 Min Late,02 Hrs 32 Mins Late,11:36 AM,10:12 AM
9,"13 Dec, 2019",01 Min Late,No Delay,11:36 AM,07:40 AM


In [69]:
#these data weekly or monthly so we can incorporate changing if any there but we need to track alert for issues
df_date_range.to_csv('./12557_running_2016_2018.csv')